In [14]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import os

#Quick url maker
def make_url(base_url,comp):
    
    url = base_url
    
    # add each component to the base url
    for r in comp:
        url = '{}/{}'.format(url,r)
        
    return url

#Save function

def name_statement_csv(yeet):
    if 'Balance Sheets' in yeet:
        return 'Balance_Sheet.csv'
    elif 'Operations' in yeet:
        return 'Operations.csv'
    elif 'Comprehensive Loss' in yeet:
        return 'Comprensive_Loss.csv'
    elif 'Cash Flows' in yeet:
        return 'cash_flows.csv'
    elif 'BALANCE SHEETS' in yeet:
        return 'Balance_Sheet.csv'
    elif 'OPERATIONS' in yeet:
        return 'Operations.csv'
    elif 'COMPREHENSIVE LOSS' in yeet:
        return 'Comprensive_Loss.csv'
    elif 'CASH FLOWS' in yeet:
        return 'cash_flows.csv'
    else:
        return 'Other.csv'

def create_dir(base_dir,comp):

    dir_ = base_dir

    # add each component to the base url
    for r in comp:
        dir_ = '{}\{}'.format(dir_,r)

    return dir_

#XML version

CIK = '320193'

endpoint = r"https://www.sec.gov/cgi-bin/browse-edgar"

param_dict = {'action' : 'getcompany',
              'CIK' : '320193',
              'type' : '10-K',
              'dateb' : '20200421',
              'owner' : 'exclude',
              'start' : '',
              'output' : 'atom',
              'count' : '100'}

#request url then parse
response = requests.get(url = endpoint, params=param_dict)
soup = BeautifulSoup(response.content,'lxml')

entries = soup.find_all('entry')

master_list_xml = []

for entry in entries:
    
    accession_num = entry.find('accession-nunber').text

        # create a new dictionary
    entry_dict = {}
    entry_dict[accession_num] = {}
    
    # store the category info
    category_info = entry.find('category')    
    entry_dict[accession_num]['category'] = {}
    entry_dict[accession_num]['category']['label'] = category_info['label']
    entry_dict[accession_num]['category']['scheme'] = category_info['scheme']
    entry_dict[accession_num]['category']['term'] =  category_info['term']

    # store the file info
    entry_dict[accession_num]['file_info'] = {}
    entry_dict[accession_num]['file_info']['file_number'] = entry.find('file-number').text
    entry_dict[accession_num]['file_info']['file_number_href'] = entry.find('file-number-href').text
    entry_dict[accession_num]['file_info']['filing_date'] =  entry.find('filing-date').text
    entry_dict[accession_num]['file_info']['filing_href'] = entry.find('filing-href').text
    entry_dict[accession_num]['file_info']['filing_type'] =  entry.find('filing-type').text
    entry_dict[accession_num]['file_info']['form_number'] =  entry.find('film-number').text
    entry_dict[accession_num]['file_info']['form_name'] =  entry.find('form-name').text
    entry_dict[accession_num]['file_info']['file_size'] =  entry.find('size').text
    
    # store extra info
    entry_dict[accession_num]['request_info'] = {}
    entry_dict[accession_num]['request_info']['link'] =  entry.find('link')['href']
    entry_dict[accession_num]['request_info']['title'] =  entry.find('title').text
    entry_dict[accession_num]['request_info']['last_updated'] =  entry.find('updated').text
    
    # store in the master list
    master_list_xml.append(entry_dict)
    
urls = []

base_url = r"https://www.sec.gov/Archives/edgar/data"
for i in master_list_xml:
    for j in i:
        j = str(j)
        
        doc_url = make_url(base_url,[CIK,j.replace("-",""),"index.json"])
        urls.append(doc_url)
        
n = 0
for url in urls:
    content = requests.get(url).json()

    for file in content['directory']['item']:
        if file['name'] == 'FilingSummary.xml':

            xml_summary = r"https://www.sec.gov" + content['directory']['name'] + "/" + file['name']

#            print(file['name'])
            print(xml_summary)

            


    base_url = xml_summary.replace('FilingSummary.xml','')

    content = requests.get(xml_summary).content
    soup = BeautifulSoup(content,'lxml')

    reports = soup.find('myreports')
    
    master_reports = []
    
    for report in reports.find_all('report')[:-1]:
        report_dict = {}
        report_dict['name_short'] = report.shortname.text
        report_dict['name_long'] = report.longname.text
        try:
            report_dict['position'] = report.position.text
        except:
            report_dict['position'] = 'NONE'
        try:
            report_dict['category'] = report.menucategory.text
        except:
            report_dict['category'] = 'NONE'
        report_dict['url'] = base_url + report.htmlfilename.text
    
        master_reports.append(report_dict)


# create the list to hold the statement urls
    statements_url = []

    for report_dict in master_reports:
    
        # define the statements we want to look for.
        item1 = r"Balance Sheets"
        item2 = r"Operations and Comprehensive Income (Loss)"
        item3 = r"Cash Flows"
        item4 = r"Stockholder's Equity"
        item5 = r"Operations"
        item6 = r"Comprehensive Loss"
        
        ITEM1 = r"BALANCE SHEETS"
        ITEM2 = r"OPERATIONS AND COMPREHENSIVE INCOME"
        ITEM3 = r"CASH FLOWS"
        ITEM4 = r"STOCKHOLDER'S EQUITY"
        ITEM5 = r"OPERATIONS"
        ITEM6 = r"COMPREHENSIVE LOSS"

        # store them in a list.
        report_list = [item1, item2, item3, item4,item5,item6]
        REPORT_LIST = [ITEM1,ITEM2,ITEM3,ITEM4,ITEM5,ITEM6]

        for i in range(len(report_list)):    # if the short name can be found in the report list.
            if report_list[i] in report_dict['name_short']:

                statements_url.append(report_dict['url'])

            elif REPORT_LIST[i] in report_dict['name_short']:

                statements_url.append(report_dict['url'])
            
    print(statements_url)

    # let's assume we want all the statements in a single data set.
    statements_data = []

    # loop through each statement url
    for statement in statements_url:

        # define a dictionary that will store the different parts of the statement.
        statement_data = {}
        statement_data['headers'] = []
        statement_data['sections'] = []
        statement_data['data'] = []

        # request the statement file content
        content = requests.get(statement).content
        report_soup = BeautifulSoup(content, 'html')


        # find all the rows, figure out what type of row it is, parse the elements, and store in the statement file list.
        for index, row in enumerate(report_soup.table.find_all('tr')):

            # first let's get all the elements.
            cols = row.find_all('td')

            # if it's a regular row and not a section or a table header
            if (len(row.find_all('th')) == 0 and len(row.find_all('strong')) == 0): 
                reg_row = [ele.text.strip() for ele in cols]
                statement_data['data'].append(reg_row)

            # if it's a regular row and a section but not a table header
            elif (len(row.find_all('th')) == 0 and len(row.find_all('strong')) != 0):
                sec_row = cols[0].text.strip()
                statement_data['sections'].append(sec_row)

            # finally if it's not any of those it must be a header
            elif (len(row.find_all('th')) != 0):            
                hed_row = [ele.text.strip() for ele in row.find_all('th')]
                statement_data['headers'].append(hed_row)

            else:            
                print('We encountered an error.')

        # append it to the master list.
        statements_data.append(statement_data)

    




    # Grab the proper components

    for statement in range(len(statements_data)):
        try:
            income_header =  statements_data[statement]['headers'][1]
        except:
            income_header = statements_data[statement]['headers'][0][-2:]

        income_data = statements_data[statement]['data']
        category = statements_data[statement]['headers'][0][0]

        # Put the data in a DataFrame
        income_df = pd.DataFrame(income_data)


        # Define the Index column, rename it, and we need to make sure to drop the old column once we reindex.
        income_df.index = income_df[0]
        income_df.index.name = category.replace("$","").replace("()","")
        income_df = income_df.drop(0, axis = 1)

        # Get rid of the '$', '(', ')', and convert the '' to NaNs.
        income_df = income_df.replace('[\$,)]','', regex=True )\
                             .replace( '[(]','-', regex=True)\
                             .replace( '', 'NaN', regex=True)

        # everything is a string, so let's convert all the data to a float.
        income_df = income_df.astype(float)

        # Change the column headers
        income_df.columns = income_header
    


        for key in master_list_xml[n]:
            key = key

        date = master_list_xml[n][str(key)]['file_info']['filing_date']
        
        u = r"C:\Notebooks\Financials Data\Company"
        r = [CIK]
        if not os.path.exists(create_dir(u,r)):
            os.mkdir(create_dir(u,r))
            
        print(create_dir(u,r))
        u = r"C:\Notebooks\Financials Data\Company"
        r = [CIK,date]
        if not os.path.exists(create_dir(u,r)):
            os.mkdir(create_dir(u,r))

        u = r"C:\Notebooks\Financials Data\Company"
        r = [CIK,date,name_statement_csv(category)]
        income_df.to_csv(create_dir(u,r))
        
    n += 1
    

https://www.sec.gov/Archives/edgar/data/320193/000032019319000119/FilingSummary.xml
['https://www.sec.gov/Archives/edgar/data/320193/000032019319000119/R2.htm', 'https://www.sec.gov/Archives/edgar/data/320193/000032019319000119/R4.htm', 'https://www.sec.gov/Archives/edgar/data/320193/000032019319000119/R5.htm', 'https://www.sec.gov/Archives/edgar/data/320193/000032019319000119/R7.htm', 'https://www.sec.gov/Archives/edgar/data/320193/000032019319000119/R55.htm', 'https://www.sec.gov/Archives/edgar/data/320193/000032019319000119/R60.htm', 'https://www.sec.gov/Archives/edgar/data/320193/000032019319000119/R70.htm']
C:\Notebooks\Financials Data\Company\320193
C:\Notebooks\Financials Data\Company\320193
C:\Notebooks\Financials Data\Company\320193
C:\Notebooks\Financials Data\Company\320193
C:\Notebooks\Financials Data\Company\320193
C:\Notebooks\Financials Data\Company\320193
C:\Notebooks\Financials Data\Company\320193
https://www.sec.gov/Archives/edgar/data/320193/000032019318000145/Filing

ValueError: could not convert string to float: '[1]'

In [13]:
list = [r"Balance Sheets"]

if list[0] in "Consolidated Balance Sheets":
    print('yeet')
elif "Consolidated Balance Sheets" in list:
    print('yeet')
else:
    print("no yeet")


yeet


1